In [1]:
import requests
import csv
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
base_url = "https://watchseriestv.top/movie?page="
num_page = 3

In [3]:
movie_data = []
#create url for each page 
for page_num in range(1, num_page+1):
    url = base_url + str(page_num)
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html")

    #access to each movie site
    movie_links = soup.find_all("a", class_="film-poster-ahref flw-item-tip")
    for link in movie_links:
        link = "https://watchseriestv.top/" + link["href"]
        movie_response = requests.get(link)
        movie_soup = BeautifulSoup(movie_response.text, "html.parser")
        
        #info movie
        movie_title = movie_soup.find("h2", class_="heading-name").text
        movie_description = movie_soup.find("div", class_="description").text.replace("\n", "").strip()
        
        row_line = movie_soup.find_all("div", class_="row-line")
        movie_release_date = row_line[0].text.replace("\n", "").split(':  ')[-1].strip()
        movie_genres = [genre.text for genre in row_line[1].find_all("a")]
        movie_actors = [actor.text for actor in row_line[2].find_all("a")]
        movie_duration = row_line[3].text.replace("\n","").split("  ")[1].strip()
        movie_country = row_line[4].text.split('\n')[2]       
        movie_productions = [production.text for production in row_line[5].find_all("a")] 
        movie_img_url = movie_soup.find("img", class_="film-poster-img")["src"]

        #add movie_data list 
        movie_data.append({'title':movie_title, 'description':movie_description, 'release_date':movie_release_date, 'genres':movie_genres, 'actors':movie_actors, 'duration':movie_duration, 'country':movie_country, 'productions':movie_productions, 'img_url':movie_img_url})


In [12]:
movie_data = pd.DataFrame(movie_data)
movie_data.to_csv('movies.csv')
